# Chapter 4: Training  Models


## <font color=blue>Linear Regression</font>

a linear model makes a predictipon by simply computing a weighted sum of the input features, plus a constant called the bias term:

$$\hat{y}=\theta_{0}+\theta_{1} x_{1}+\theta_{2} x_{2}+\cdots+\theta_{n} x_{n}$$

$$\hat{y}=h_{\theta}(\mathbf{x})=\boldsymbol{\theta} \cdot \mathbf{x}$$

### Normal Equation

$$\widehat{\boldsymbol{\theta}}=\left(\mathbf{X}^{\top} \mathbf{X}\right)^{-1} \mathbf{X}^{\top} \mathbf{y}$$

- This equation gives us the weights directly
- pseudo inverse can be used instead of inverse, as $\left(\mathbf{X}^{\top} \mathbf{X}\right)$ might be singular, as result of redundant features or not enough instances


scikit learn LinearRegression class uses normal equations and is based on scipy.linalg.lstsq() which computes pseudo inverse using SVD.

computing inverse of this matrix has a computational complexity of $O(n^2.4)$ to $O(n^3)$ depending on the implementation. scikit learn approach's complexity is about $O(n^2)$.

normal equation and svd method get very slow as numver of features ($n$) grows. On the positive side they are linear regarding to the number of instances.

## <font color=blue> Gradient Descent</font>

Gradient Descent algorithm tweaks parameters iteratively in order to minimize a cost function.

### Batch Gradient Descent

batch gradient descent computes the partial derivative if the cost function with regard to parameter $\theta_j$ :
$$\frac{\partial}{\partial \theta_{j}} \operatorname{MSE}(\boldsymbol{\theta})=\frac{2}{m} \sum_{i=1}^{m}\left(\boldsymbol{\theta}^{\top} \mathbf{x}^{(i)}-y^{(i)}\right) x_{j}^{(i)}$$

We can re write this in the form below to compute all the derivatives in one go.The gradient vector contains all the derivatives of the cost function:
$$\nabla_{\theta} \operatorname{MSE}(\boldsymbol{\theta})=\left(\begin{array}{c}
\frac{\partial}{\partial \theta_{0}} \operatorname{MSE}(\boldsymbol{\theta}) \\
\frac{\partial}{\partial \theta_{1}} \operatorname{MSE}(\boldsymbol{\theta}) \\
\vdots \\
\frac{\partial}{\partial \theta_{\mathrm{n}}} \operatorname{MSE}(\boldsymbol{\theta})
\end{array}\right)=\frac{2}{m} \mathbf{X}^{\top}(\mathbf{X} \boldsymbol{\theta}-\mathbf{y})$$

- This method computes the derivatives over the whole training set $\mathbf{X}$ at each step. this is why it's called Batch Gradient Descent. As a result it gets slow on very large training sets. However, Gradient Descent scales well with the number of features.

Once we have the gradient vector, we can calculate new weights:
$$\boldsymbol{\theta}^{(\text {next step })}=\boldsymbol{\theta}-\eta \nabla_{\boldsymbol{\theta}} \operatorname{MSE}(\boldsymbol{\theta})$$ where $\eta$ is the learning rate

### Stochastic Gradient Descent

Stochastic Gradient Descent, unlike Batch Gradient Descent, picks a random instance in training set at every training step and computes gradients based on that single instance. Over time it will get close to the minimum, but once it gets there it will continue to bounce around, never settling down. So once the algorithm stops, the final parameter values are good, but not optimal.

Pros:
- faster convergence
- more likely to find global minimum
- can be implemented as an out-of-core algorithm

While the randomness is good for escaping from local optima by jumping over, it's bad as it can never settle at the minimum. one solution to this problem is to gradually reduce the learning rate. the learning rate should not be reduced too quickly or too slowly. The function that determines the learning rate at each iteration is called the learning schedule.

In SGD implementation, epochs are used instead of iterations. While BGD calculates over the whole data set in every iteration, SGD calculates derivatives over every instance separately in every epoch; so it goes through the training set on each epoch.

When using SGD, the training instances must be independent and identically disturbed to ensure that the parameters get pulled toward the global optimum. A simple way to ensure this is to shuffle the instances
during training (e.g., pick each instance randomly, or shuffle the training set at the
beginning of each epoch). If you do not shuffle the instances—for example, if the
instances are sorted by label—then SGD will start by optimizing for one label, then the
next, and so on, and it will not settle close to the global minimum.


Scikit-Learn has implemented SGD in SGDRegressor class.

### Mini-Batch Gradient Descent

Mini-Batch GD, instead of training on whole training set as in BGD or training on one instance as in SGD on each step, computes gradients on a small random set of instances. Mini-Batch GD benefits from hardware optimization of matrix operations, especially when using GPUs. It will end up walking around a bit closer to the minimum than
Stochastic GD—but it may be harder for it to escape from local minima.

$$\begin{array}{llllll}
\hline \text { Algorithm } & \begin{array}{l}
\text { Large } \\
m
\end{array} & \begin{array}{l}
\text { Out-of-core } \\
\text { support }
\end{array} & \begin{array}{l}
\text { Large } \\
\boldsymbol{n}
\end{array} & \text { Hyperparams } & \begin{array}{l}
\text { Scaling } \\
\text { required }
\end{array} & \text { Scikit-Learn } \\
\hline \begin{array}{l}
\text { Normal } \\
\text { Equation }
\end{array} & \text { Fast } & \text { No } & \text { Slow } & 0 & \text { No } & \text { N/A } \\
\hline \text { SVD } & \text { Fast } & \text { No } & \text { Slow } & 0 & \text { No } & \text { LinearRegression } \\
\hline \text { Batch GD } & \text { Slow } & \text { No } & \text { Fast } & 2 & \text { Yes } & \text { SGDRegressor } \\
\hline \begin{array}{l}
\text { Stochastic } \\
\text { GD }
\end{array} & \text { Fast } & \text { Yes } & \text { Fast } & \geq 2 & \text { Yes } & \text { SGDRegressor } \\
\hline \begin{array}{l}
\text { Mini-batch } \\
\text { GD }
\end{array} & \text { Fast } & \text { Yes } & \text { Fast } & \geq 2 & \text { Yes } & \text { SGDRegressor }
\end{array}$$

## <font color=blue> Polynomial Learning</font>

We can use linear regression models to train non linear data by adding combinations and higher degrees of features. This can be done with scikit-learn PloynomialFeatures class from <c>preprocessing</c> package. Note that when there are multiple features, Polynomial Regression is capable
of finding relationships between features (which is something a plain Linear
Regression model cannot do). This is made possible by the fact that
PolynomialFeatures also adds all combinations of features up to the given
degree. 

## <font color=blue>Learning Curves</font>

One way to see if a model is over-fitting or not was to use cross-validation metrics. Another way to tell is to look at the learning curves of a model; these are plots of model performance on the training set and validation set as function of the training set size or the training iteration (checkout training_models notebook for examples).
generally the train error increases as size of the training set grows, but validation error decreases. Eventually these two get close to each other and end up in plateau and don't change as the training set gets bigger. If both curves reach a plateau close to each other and relatively high; the model is probably under-fitting. On the other hand if plateaus are far from each other (low train error and high validation error) the model is over-fitting.


### <font color=red>Bias-Variance Trade-off</font>

A model's generalization error can be expressed as sum of three errors:
- #### Bias
this part of error is due to wrong assumptions, such as assuming that data is linear when it's actually quadratic. A high-bias model is likely to under-fit the training data.
- #### Variance
This part is due to the model’s excessive sensitivity to small
variations in the training data. A model with many degrees of
freedom (such as a high-degree polynomial model) is likely to have
high variance and thus overfit the training data.
- #### Irreducible error
This part is due to the noisiness of the data itself.

Increasing a model’s complexity will typically increase its variance and reduce its bias. Conversely, reducing a model’s complexity increases its bias and reduces its variance. This is why it is called a trade-off.


## <font color=blue>Regularized Linear Models</font>

### Ridge Regression

Ridge regression is a regularized version of linear regression which a regularization term equal to $\alpha \sum_{i=1}^{n} \theta_{i}^{2}$ is added to cost function. The hyperparameter $\alpha$ controls how much you want to regularize the model.
$$J(\boldsymbol{\theta})=\operatorname{MSE}(\boldsymbol{\theta})+\alpha \frac{1}{2} \sum_{i=1}^{n} \theta_{i}^{2}$$
Note that bias therm $\theta_0$ is not regularized. 

If we define $\mathbf{w}$ as the vector of feature weights (θ1 to θn), then the
regularization term is equal to $1 / 2\left(\|\mathbf{w}\|_{2}\right)^{2},$ where $\|\mathbf{w}\|_{2}$represents the $\ell_{2}$ norm of the weight vector. For Gradient Descent, just add $\alpha\mathbf{w}$ to the $\operatorname{MSE}$ gradient vector.


- #### WARNING
It is important to scale the data before performing Ridge Regression, as it is sensitive to the scale of the input features.

Ridge Regression can be implemented in both closed form using a matrix factorization technique (Cholesky) and Gradient Descent.<br>
Closed Form method can show in the following form in which $\mathbf{A}$ is the $(n+1)×(n+1)$ identity matrix, except with a 0 in the top-left cell. corresponding to the bias term:
$$\widehat{\boldsymbol{\theta}}=\left(\mathbf{X}^{\top} \mathbf{X}+\alpha \mathbf{A}\right)^{-1} \mathbf{X}^{\top} \mathbf{y}$$
Ridge Regression can be performed with scikit-learn Ridge class in linear_models package with solver="cholesky" parameter; and using SGD with penalty="l2" parameter.

### Lasso Regression

<i>Least Absolute Shrinkage and Selection Operator Regression</i> or Lasso Regression, just like Ridge Regression, adds a regularization term to cost function; but it uses $\ell_1$ norm of the weights.
$$J(\boldsymbol{\theta})=\operatorname{MSE}(\boldsymbol{\theta})+\alpha \sum_{i=1}^{n}\left|\theta_{i}\right|$$

An important characteristic of Lass Regression is that it tends to eliminate unimportant features by setting their weights to zero. Actually Lasso performs an automatic feature selection and outputs a sparse model. checkout figure 4-19 of book for more insights.

Note that in Ridge Regression, gradients get smaller as the parameters get closer to global optimum, so gradient descent slows down which helps convergence, while with lasso regression, gradient descent bounces around the optimum. Also features with less importance get smaller as $\alpha$ increases, but they never get eliminated entirely as in Lasso Regression.

Keep in mind that Lasso cost function is not differentiable at $\theta_i = 0$, but gradient descent still works fine if we use a sub-gradient vector $\mathbf{g}$ instead when any $\theta_i = 0$.
$$g(\boldsymbol{\theta}, J)=\nabla_{\boldsymbol{\theta}} \operatorname{MSE}(\boldsymbol{\theta})+\alpha\left(\begin{array}{c}
\operatorname{sign}\left(\theta_{1}\right) \\
\operatorname{sign}\left(\theta_{2}\right) \\
\vdots \\
\operatorname{sign}\left(\theta_{n}\right)
\end{array}\right) \quad \text { where } \operatorname{sign}\left(\theta_{i}\right)=\left\{\begin{array}{cl}
-1 & \text { if } \theta_{i}<0 \\
0 & \text { if } \theta_{i}=0 \\
+1 & \text { if } \theta_{i}>0
\end{array}\right.$$

Lasso Regression is available through Lasso class in linear_model package from Scikit-Learn. We can also use SGDResgressor with penalty="l1" parameter instead.

### Elastic Net

Elastic net is a middle ground between Ridge and Lasso Regression. The regularization term is a simple mix of both Ridge and
Lasso’s regularization terms, and you can control the mix ratio $r$.
$$J(\boldsymbol{\theta})=\operatorname{MSE}(\boldsymbol{\theta})+r \alpha \sum_{i=1}^{n}\left|\theta_{i}\right|+\frac{1-r}{2} \alpha \sum_{i=1}^{n} \theta_{i}^{2}$$

Elastic Net is accessible through sickit learn ElasticNet class.

So when should you use plain Linear Regression (i.e., without any
regularization), Ridge, Lasso, or Elastic Net? It is almost always preferable
to have at least a little bit of regularization, so generally you should avoid
plain Linear Regression. Ridge is a good default, but if you suspect that only
a few features are useful, you should prefer Lasso or Elastic Net because
they tend to reduce the useless features’ weights down to zero, as we have
discussed. In general, Elastic Net is preferred over Lasso because Lasso may
behave erratically when the number of features is greater than the number of
training instances or when several features are strongly correlated.


### Early Stopping

A very different way to regularize iterative learning algorithms such as
Gradient Descent is to stop training as soon as the validation error reaches a
minimum.  As the epochs go by the algorithm learns, and
its prediction error (RMSE) on the training set goes down, along with its
prediction error on the validation set. After a while though, the validation
error stops decreasing and starts to go back up. This indicates that the model
has started to overfit the training data. With early stopping you just stop
training as soon as the validation error reaches the minimum.

## <font color=blue>Logistic Regression</font>

Some regression algorithms can be used for classification. Logistic Regression (also called Logit
Regression) is commonly used to estimate the probability that an instance
belongs to a particular class (e.g., what is the probability that this email is
spam?). If the estimated probability is greater than 50%, then the model
predicts that the instance belongs to that class (called the positive class,
labeled “1”), and otherwise it predicts that it does not (i.e., it belongs to the
negative class, labeled “0”). This makes it a binary classifier.


### Estimating Probabilities

A Logistic Regression model computes a weighted sum of the input features just like a Linear Regression model, but instead of outputting the result directly, it outputs the logistic of the result.
$$\hat{p}=h_{\boldsymbol{\theta}}(\mathbf{x})=\sigma\left(\mathbf{x}^{\top} \boldsymbol{\theta}\right)$$
The logistic (noted $\sigma$) is a sigmoid function that outputs a number between 0 and 1.
$$\sigma(t)=\frac{1}{1+\exp (-t)}$$

Once the Logistic Regression model has estimated the probability $\hat{p}=h_{\boldsymbol{\theta}}(\mathbf{x})$
that an instance x belongs to the positive class, it can make its prediction ŷ
easily.
$$\hat{y}=\left\{\begin{array}{ll}
0 & \text { if } \hat{p}<0.5 \\
1 & \text { if } \hat{p} \geq 0.5
\end{array}\right.$$

### Training and Cost Function

$$c(\boldsymbol{\theta})=\left\{\begin{array}{cl}
-\log (\hat{p}) & \text { if } y=1 \\
-\log (1-\hat{p}) & \text { if } y=0
\end{array}\right.$$

This cost function makes sense because $-\log (t)$ grows very large when approaches $0,$ so the cost will be large if the model estimates a probability close to 0 for a positive instance, and it will also be very large if the model estimates a probability close to 1 for a negative instance. On the other hand, $\log (t)$ is close to 0 when $t$ is close to $1,$ so the cost will be close to 0 if the estimated probability is close to 0 for a negative instance or close to 1 for a positive instance, which is precisely what we want.

The cost function over the whole training set is the average cost over all
training instances. It can be written in a single expression called the log loss,
shown in Equation 4-17.
$$J(\boldsymbol{\theta})=-\frac{1}{m} \sum_{i=1}^{m}\left[y^{(i)} \log \left(\hat{p}^{(i)}\right)+\left(1-y^{(i)}\right) \log \left(1-\hat{p}^{(i)}\right)\right]$$

This cost function has no closed form equations to compute the value of $\theta$ that minimizes the cost function. But it is a convex function so gradient descent can always find the global minimum.<br>
The partial derivatives of the cost function with regard to the $j^{\text {th }}$ model parameter $\theta_{j}$ are given by Equation 4-18.
$$\frac{\partial}{\partial \theta_{j}} \mathrm{J}(\boldsymbol{\theta})=\frac{1}{m} \sum_{i=1}^{m}\left(\sigma\left(\boldsymbol{\theta}^{\top} \mathbf{x}^{(i)}\right)-y^{(i)}\right) x_{j}^{(i)}$$
This equation looks very much like the gradients equation for MSE cost function. It can be used with BGD, SGD or Mini-Bath GD.

### Decision Boundaries

In this section we'll use Logistic Regression on Iris dataset to detect Iris Virginica type based only on the petal width feature.

Not in the mood for it! Read the book.

### Softmax Regression

As we saw in chapter 3 we can use multiple binary classifiers to solve multi-classs classification problems. Softmax regression generalizes Logisitic Regression so that it can be used for multiple classes without having to traing multiple binary classifiers.

In Softmax Regression there is a weight vector for each target class. so we have a $k\times n$ weights matrix, where $k$ is the number of target classes and $n$ is the number of features. to predict class of a input vector $\mathbf{X}$, we should compute a score for each class. this score is defined as follows:
$$s_{k}(\mathbf{x})=\mathbf{x}^{\top} \boldsymbol{\theta}^{(k)}$$
where $\boldsymbol{\theta}^{(k)}$ is the weight vector for the $\mathbf{k}_{th}$ class.<br>
After computing score for every class, we can compute the probability $\hat{p}_{k}$ that the instance belongs to class $\mathbf{k}$ by running the scores through the softmax function, which computes the exponential of every score, then normalizes them. The scores are called <i>logits</i> or <i>log-odds</i>.<br><i>Softmax function:</i>
$$\hat{p}_{k}=\sigma(\mathbf{s}(\mathbf{x}))_{k}=\frac{\exp \left(s_{k}(\mathbf{x})\right)}{\sum_{j=1}^{K} \exp \left(s_{j}(\mathbf{x})\right)}$$

In this equation:
- $K$ is the number of classes.
- $\mathbf{s}(\mathbf{x})$ is a vector containing the scores of each class for the instance $\mathbf{x} .$
- $\sigma(\mathbf{s}(\mathbf{x}))_{k}$ is the estimated probability that the instance $\mathbf{x}$ belongs to class $k$, given the scores of each class for that instance.

The class with highest probability will be predicted as the target class:
$$\hat{y}=\underset{k}{\operatorname{argmax}} \sigma(\mathbf{s}(\mathbf{x}))_{k}=\underset{k}{\operatorname{argmax}} s_{k}(\mathbf{x})=\underset{k}{\operatorname{argmax}}\left(\left(\boldsymbol{\theta}^{(k)}\right)^{\top} \mathbf{x}\right)$$

We must also train the model as well. The objective is to have a model that estimates a high probability for target class and a low prob. for other classes. Minimizing cost function in equation $22$, called the <i>cross entropy</i>, should lead to this objective.
$$J(\boldsymbol{\Theta})=-\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_{k}^{(i)} \log \left(\hat{p}_{k}^{(i)}\right)$$
In this equation, $y_K^{(i)}$ is either equal to 1 or 0, depending on whether the
instance belongs to the class or not.


Notice that when there are just two classes (K = 2), this cost function is
equivalent to the Logistic Regression's cost function (eq.17)

The gradient vector for this cost function with regard to $\boldsymbol{\theta}^{(k)}$ is estimated as follows:
$$\nabla_{\boldsymbol{\theta}^{(k)}} J(\boldsymbol{\Theta})=\frac{1}{m} \sum_{i=1}^{m}\left(\hat{p}_{k}^{(i)}-y_{k}^{(i)}\right) \mathbf{x}^{(i)}$$

Scikit-learn's LogisticRegression use one-versus-the-rest by default when trained on more than two classes, but we can set multi_class hyperparameter to "multinomial" to use Softmax Regression. The solver also must be set to sth that supports softmax, such as "lbfgs". it also applies $\ell_2$ regularization.